## Análisis del modelo matematico propuesto

In [70]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#  Cargar los datos del archivo CSV
df_SFV = pd.read_csv('../Data/Cleaned/SFV_2023.csv', sep=',')
df_solcast = pd.read_csv('../Data/Cleaned/solcast_2023.csv', sep=',')

In [76]:
#  Cargar los datos del archivo CSV
Fecha = '2023-05-05'
df_SFV = pd.read_csv('../Data/Cleaned/SFV_2023.csv', sep=',')
df_solcast = pd.read_csv('../Data/Cleaned/solcast_2023.csv', sep=',')
row_day_solcast = df_solcast.loc[df_solcast['Date'] == Fecha]
row_day_fronius = df_SFV.loc[df_SFV['Date'] == Fecha]
# reiniciar el index
row_day_solcast = row_day_solcast.reset_index(drop=True)
row_day_fronius = row_day_fronius.reset_index(drop=True)
# Odenernar row_day_solcast de menor a mayor hora
row_day_solcast = row_day_solcast.sort_values(by='Time')
row_day_solcast = row_day_solcast.reset_index(drop=True)
# Crear un nuevo dataframe con los datos de solcast y fronius a partir del indice 
df = pd.DataFrame()
df['Time'] = row_day_solcast['Time']
df['GHI'] = row_day_solcast['GHI']
df['air_temp'] = row_day_solcast['air_temp']
df['cloud_opacity'] = row_day_solcast['cloud_opacity']
df['Producción_fotovoltaica_SFV'] = row_day_fronius['Producción_fotovoltaica_SFV']

In [77]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

#  Cargar los datos del archivo CSV
import ipywidgets as widgets
from IPython.display import display

def plot_signal(alpha, beta):
    #Parametros SFV
    efficiency = 0.2094  # Eficiencia del panel solar
    catidad_pv= 30
    area =2.274*1.134
    
    # normalizar 'cloud_opacity' de 0 a 1  en donde 1 es el valor más alto
    df['cloud_opacity'] = df['cloud_opacity'] / df['cloud_opacity'].max()
    # normalizar 'air_temp' de 0 a 1  en donde 1 es el valor más alto
    df['air_temp'] = df['air_temp'] / df['air_temp'].max()
    
    
    # Energía calculada a partir de Solcast 
    df['P_cloud_opacity_temp'] = df['GHI'] * efficiency *  area * catidad_pv * (1 - beta*df['cloud_opacity']) * (1 - alpha*df['air_temp'])

    # Energía calculada a partir de Fronius
    df['Potencia'] = df['Producción_fotovoltaica_SFV'] / (5/60)
    
    plt.figure(figsize=(15, 4))
    plt.plot(df['Time'], df['P_cloud_opacity_temp'], label='Solcast')
    plt.plot(df['Time'], df['Potencia'], label='Fronius')
    # grafico area bajo la curva
    plt.fill_between(df['Time'], df['P_cloud_opacity_temp'], df['Potencia'], color='gray', alpha=0.2)
    plt.title(f'Producción fotovoltaica calculada de {Fecha} con: Alpha={alpha}, Beta={beta}  \nP = Irr * efficiency *  area * catidad_pv * (1-beta * N) * (1-alpha*T)')
    plt.xlabel('Tiempo')
    plt.ylabel('Amplitud')
    plt.legend()
    plt.xticks([100, 150, 200])
    # plt.grid(True)
    plt.show()
    # Energía calculada a partir de Fronius
    energia_fronius = df['Producción_fotovoltaica_SFV'].sum()
    # Horas de brillo solar
    hbs = energia_fronius / (30*0.54)
    print("Horas de brillo solar =", hbs/1000)
    print("Producción fotovoltaica SVF [kWh/día] =",energia_fronius/1000)
    # suma de la potencia generada solcast
    energia = df['P_cloud_opacity_temp']*(5/60)
    print("Energía calculada solacast [kWh] =",energia.sum()/1000)
    # error porcentual
    error = abs(  energia.sum() - energia_fronius.sum()) / energia_fronius.sum() * 100
    print('Error porcentual:', error, '%')

# Definir sliders
amplitude_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='alpha:')
frequency_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='beta:')

# Crear la interfaz interactiva
interact(plot_signal, alpha=amplitude_slider, beta=frequency_slider)

interactive(children=(FloatSlider(value=0.5, description='alpha:', max=1.0), FloatSlider(value=0.5, descriptio…

<function __main__.plot_signal(alpha, beta)>